# Asian Option Pricing 


## 1 — Imports
Import required Python libraries.

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import math



## 2 — Parameters 
Core model parameters (spot, strike, rate, vol, maturity, etc.).

In [45]:
S0    = 1235.0       # initial spot
K     = 1800.0      # strike
r     = 0.12      # risk-free rate
sigma = 0.20      # volatility
T     = 1.0        # maturity (years)
n_obs = 12         # observation count (discrete averaging)
n_paths = 20000    # Monte Carlo paths (reduce if slow)
seed = 2025

print('Parameters set: S0={}, K={}, r={}, sigma={}, T={}, n_obs={}, n_paths={}'.format(
    S0, K, r, sigma, T, n_obs, n_paths))

Parameters set: S0=1235.0, K=1800.0, r=0.12, sigma=0.2, T=1.0, n_obs=12, n_paths=20000


## 3 — Observation times


In [46]:
times = np.linspace(T / n_obs, T, n_obs)
dt = np.diff(np.concatenate(([0.0], times)))
sqrt_dt = np.sqrt(dt)
print('Times:', times)
print('dt:', dt)

Times: [0.08333333 0.16666667 0.25       0.33333333 0.41666667 0.5
 0.58333333 0.66666667 0.75       0.83333333 0.91666667 1.        ]
dt: [0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333]


## 4 — Geometric-average Asian (analytic background)
The geometric-average Asian option has a closed-form under Black-Scholes because the log of the geometric average is normal. We'll implement the analytic formula for discrete sampling.

In [47]:
def geometric_asian_call_price(S0, K, r, sigma, times):
   
    times = np.array(times, dtype=float)
    n = len(times)
    if n == 0:
        return 0.0

    # mean log S at observation times
    m_i = np.log(S0) + (r - 0.5 * sigma**2) * times
    muG = m_i.mean()

    # var(log G)
    Tmat = np.minimum.outer(times, times)
    varG = (sigma**2 / (n**2)) * Tmat.sum()

    if varG <= 0:
        g0 = np.exp(muG)
        payoff = max(g0 - K, 0.0)
        return math.exp(-r * times[-1]) * payoff

    sqrt_varG = math.sqrt(varG)
    a = (muG - math.log(K)) / sqrt_varG
    b = (muG + varG - math.log(K)) / sqrt_varG

    EG_gt = math.exp(muG + 0.5 * varG) * norm.cdf(b)
    PG_gt = norm.cdf(a)

    price = math.exp(-r * times[-1]) * (EG_gt - K * PG_gt)
    return float(price)



### Compute the analytic geometric price (for control variate).

In [48]:
geo_price = geometric_asian_call_price(S0, K, r, sigma, times)
print('Analytic geometric Asian price:', geo_price)

Analytic geometric Asian price: 0.27042802276115235


## 5 — Monte Carlo plan (overview)
We'll simulate price paths at the observation times, compute arithmetic and geometric averages, discount payoffs, and then apply variance reduction:

- Antithetic variates
- Control variate using the analytic geometric price.

In [49]:
np.random.seed(seed)
half = n_paths // 2
print('Using {} paths ({} antithetic pairs)'.format(n_paths, half))

Using 20000 paths (10000 antithetic pairs)


### Generate standard normal variates for antithetic sampling.

In [50]:
rng = np.random.default_rng(seed)
Z_half = rng.standard_normal(size=(half, n_obs))
Z = np.vstack([Z_half, -Z_half])
print('Normals shape:', Z.shape)

Normals shape: (20000, 12)


### Precompute per-step drift and diffusion terms.

In [51]:
drift = (r - 0.5 * sigma**2) * dt
diffusion = sigma * sqrt_dt
print('drift:', drift)
print('diffusion:', diffusion)

drift: [0.00833333 0.00833333 0.00833333 0.00833333 0.00833333 0.00833333
 0.00833333 0.00833333 0.00833333 0.00833333 0.00833333 0.00833333]
diffusion: [0.05773503 0.05773503 0.05773503 0.05773503 0.05773503 0.05773503
 0.05773503 0.05773503 0.05773503 0.05773503 0.05773503 0.05773503]


### Build log-price increments and cumulative log-prices (vectorized).

In [52]:
log_increments = drift[np.newaxis, :] + diffusion[np.newaxis, :] * Z
logS = np.cumsum(log_increments, axis=1)
logS = np.log(S0) + logS
S_paths = np.exp(logS)
print('S_paths shape:', S_paths.shape)

S_paths shape: (20000, 12)


### Compute arithmetic and geometric averages across observation times for each path.

In [53]:
arith_avg = S_paths.mean(axis=1)
geo_avg = np.exp(np.mean(np.log(S_paths), axis=1))
print('Sample of arithmetic averages:', arith_avg[:5])
print('Sample of geometric averages:', geo_avg[:5])

Sample of arithmetic averages: [1001.20081627 1304.03559409 1531.27772485 1225.12050987 1604.24970327]
Sample of geometric averages: [ 998.81105811 1301.34839236 1526.8486731  1220.71318122 1594.63687607]


### Compute discounted payoffs for arithmetic and geometric payoffs.

In [54]:
discount = math.exp(-r * times[-1])
payoffs = discount * np.maximum(arith_avg - K, 0.0)
payoffs_geo = discount * np.maximum(geo_avg - K, 0.0)
print('Mean raw payoff (arith):', np.mean(payoffs))
print('Mean geo payoff:', np.mean(payoffs_geo))

Mean raw payoff (arith): 0.44186068643238213
Mean geo payoff: 0.2924137451114216


### Control variate: estimate coefficient b_hat and adjust payoffs.
b_hat = cov(payoff, pay_geo) / var(pay_geo). Then adjusted = payoff - b_hat*(pay_geo - E[pay_geo]).

In [55]:
cov_pg = np.cov(payoffs, payoffs_geo, bias=True)[0,1]
var_geo = np.var(payoffs_geo)
b_hat = cov_pg / var_geo if var_geo > 0 else 0.0
print('Estimated b_hat =', b_hat)

# analytic geo expectation already computed as geo_price
print('Analytic geo price (control):', geo_price)
adjusted = payoffs - b_hat * (payoffs_geo - geo_price)
price_est = adjusted.mean()
stderr = adjusted.std(ddof=1) / math.sqrt(len(adjusted))
print('Adjusted MC price:', price_est, 'SE:', stderr)

Estimated b_hat = 1.261068451909139
Analytic geo price (control): 0.27042802276115235
Adjusted MC price: 0.41413518558402396 SE: 0.010568542354964844


## 6 — Base case result
Summary of the base-case Monte Carlo price (with variance reduction).

In [56]:
base_row = {'Scenario': 'Base', 'S0': S0, 'r': r, 'sigma': sigma, 'Price': price_est, 'SE': stderr}
print(base_row)

{'Scenario': 'Base', 'S0': 1235.0, 'r': 0.12, 'sigma': 0.2, 'Price': 0.41413518558402396, 'SE': 0.010568542354964844}


## 7 — Stress scenarios (overview)
We'll run: vol +/-100bps, rate +/-100bps, S0 +/-5%, and random-path vol (σ per path ~ N(σ, 0.01)).

### Monte Carlo routine for a single (S0, r, sigma) triple
We implement a short inline routine to re-run MC with given parameters.

In [57]:
def mc_once(S0_, r_, sigma_, seed_local=seed, use_antithetic=True):
    rng_local = np.random.default_rng(seed_local)
    half_local = n_paths // 2
    Zh = rng_local.standard_normal(size=(half_local, n_obs))
    Z_all = np.vstack([Zh, -Zh]) if use_antithetic else np.vstack([Zh, rng_local.standard_normal(size=(half_local, n_obs))])

    drift_local = (r_ - 0.5 * sigma_**2) * dt
    diffusion_local = sigma_ * sqrt_dt

    log_inc = drift_local[np.newaxis, :] + diffusion_local[np.newaxis, :] * Z_all
    logS_local = np.cumsum(log_inc, axis=1)
    logS_local = np.log(S0_) + logS_local
    S_local = np.exp(logS_local)

    arith_avg_local = S_local.mean(axis=1)
    geo_avg_local = np.exp(np.mean(np.log(S_local), axis=1))

    discount_local = math.exp(-r_ * times[-1])
    pay_local = discount_local * np.maximum(arith_avg_local - K, 0.0)
    pay_geo_local = discount_local * np.maximum(geo_avg_local - K, 0.0)

    cov_local = np.cov(pay_local, pay_geo_local, bias=True)[0, 1]
    var_geo_local = np.var(pay_geo_local)
    b_local = cov_local / var_geo_local if var_geo_local > 0 else 0.0

    geo_analytic_local = geometric_asian_call_price(S0_, K, r_, sigma_, times)
    adjusted_local = pay_local - b_local * (pay_geo_local - geo_analytic_local)

    return float(adjusted_local.mean()), float(adjusted_local.std(ddof=1) / math.sqrt(len(adjusted_local)))

print('Helper mc_once defined')

Helper mc_once defined


### Volatility stress: run σ +100bps and σ -100bps

In [58]:
results = []
for dv in [+0.01, -0.01]:
    sigma_s = sigma + dv
    p, se = mc_once(S0, r, sigma_s, seed_local=seed)
    results.append({'Scenario': f"Vol {'+' if dv>0 else ''}{int(dv*10000)}bps", 'S0': S0, 'r': r, 'sigma': sigma_s, 'Price': p, 'SE': se})

results

[{'Scenario': 'Vol +100bps',
  'S0': 1235.0,
  'r': 0.12,
  'sigma': 0.21000000000000002,
  'Price': 0.612758960313515,
  'SE': 0.012768608661689584},
 {'Scenario': 'Vol -100bps',
  'S0': 1235.0,
  'r': 0.12,
  'sigma': 0.19,
  'Price': 0.2621891108535402,
  'SE': 0.008060072858695826}]

### Interest rate stress: run r +100bps and r -100bps

In [59]:
for dr in [+0.01, -0.01]:
    r_s = r + dr
    p, se = mc_once(S0, r_s, sigma, seed_local=seed)
    results.append({'Scenario': f"Rate {'+' if dr>0 else ''}{int(dr*10000)}bps", 'S0': S0, 'r': r_s, 'sigma': sigma, 'Price': p, 'SE': se})

results

[{'Scenario': 'Vol +100bps',
  'S0': 1235.0,
  'r': 0.12,
  'sigma': 0.21000000000000002,
  'Price': 0.612758960313515,
  'SE': 0.012768608661689584},
 {'Scenario': 'Vol -100bps',
  'S0': 1235.0,
  'r': 0.12,
  'sigma': 0.19,
  'Price': 0.2621891108535402,
  'SE': 0.008060072858695826},
 {'Scenario': 'Rate +100bps',
  'S0': 1235.0,
  'r': 0.13,
  'sigma': 0.2,
  'Price': 0.4705707352602425,
  'SE': 0.011171610927095822},
 {'Scenario': 'Rate -100bps',
  'S0': 1235.0,
  'r': 0.11,
  'sigma': 0.2,
  'Price': 0.3635532861617817,
  'SE': 0.009857702579214474}]

### Spot shock: S0 +5% and S0 -5%

In [60]:
for ds in [+0.05, -0.05]:
    S0_s = S0 * (1.0 + ds)
    p, se = mc_once(S0_s, r, sigma, seed_local=seed)
    results.append({'Scenario': f"S0 {'+' if ds>0 else ''}{int(ds*100)}%", 'S0': S0_s, 'r': r, 'sigma': sigma, 'Price': p, 'SE': se})

results

[{'Scenario': 'Vol +100bps',
  'S0': 1235.0,
  'r': 0.12,
  'sigma': 0.21000000000000002,
  'Price': 0.612758960313515,
  'SE': 0.012768608661689584},
 {'Scenario': 'Vol -100bps',
  'S0': 1235.0,
  'r': 0.12,
  'sigma': 0.19,
  'Price': 0.2621891108535402,
  'SE': 0.008060072858695826},
 {'Scenario': 'Rate +100bps',
  'S0': 1235.0,
  'r': 0.13,
  'sigma': 0.2,
  'Price': 0.4705707352602425,
  'SE': 0.011171610927095822},
 {'Scenario': 'Rate -100bps',
  'S0': 1235.0,
  'r': 0.11,
  'sigma': 0.2,
  'Price': 0.3635532861617817,
  'SE': 0.009857702579214474},
 {'Scenario': 'S0 +5%',
  'S0': 1296.75,
  'r': 0.12,
  'sigma': 0.2,
  'Price': 1.2461046753159517,
  'SE': 0.014518055649042497},
 {'Scenario': 'S0 -5%',
  'S0': 1173.25,
  'r': 0.12,
  'sigma': 0.2,
  'Price': 0.11048867244945139,
  'SE': 0.005629658142399387}]

### Random-path-vol: draw per-path sigma ~ N(σ, 0.01) and run vectorized simulation with antithetic pairing

In [61]:
rng_rpv = np.random.default_rng(seed + 999)
half_local = n_paths // 2
Z_half = rng_rpv.standard_normal(size=(half_local, n_obs))
Z_rpv = np.vstack([Z_half, -Z_half])

sigma_paths_half = rng_rpv.normal(loc=sigma, scale=0.01, size=half_local)
sigma_paths = np.concatenate([sigma_paths_half, sigma_paths_half])

# build matrices
DRIFT = (r - 0.5 * (sigma_paths[:, None]**2)) * dt[np.newaxis, :]
DIFF = (sigma_paths[:, None]) * sqrt_dt[np.newaxis, :]

log_increments_rpv = DRIFT + DIFF * Z_rpv
logS_rpv = np.cumsum(log_increments_rpv, axis=1)
logS_rpv = np.log(S0) + logS_rpv
S_paths_rpv = np.exp(logS_rpv)

arith_avg_rpv = S_paths_rpv.mean(axis=1)
geo_avg_rpv = np.exp(np.mean(np.log(S_paths_rpv), axis=1))

discount_rpv = math.exp(-r * times[-1])
pay_rpv = discount_rpv * np.maximum(arith_avg_rpv - K, 0.0)
pay_geo_rpv = discount_rpv * np.maximum(geo_avg_rpv - K, 0.0)

cov_rpv = np.cov(pay_rpv, pay_geo_rpv, bias=True)[0, 1]
var_geo_rpv = np.var(pay_geo_rpv)
b_rpv = cov_rpv / var_geo_rpv if var_geo_rpv > 0 else 0.0
geo_analytic_rpv = geometric_asian_call_price(S0, K, r, sigma, times)
adjusted_rpv = pay_rpv - b_rpv * (pay_geo_rpv - geo_analytic_rpv)

price_rpv = float(adjusted_rpv.mean())
se_rpv = float(adjusted_rpv.std(ddof=1) / math.sqrt(len(adjusted_rpv)))

results.append({'Scenario': 'Random-path-vol (σ~N(σ,0.01))', 'S0': S0, 'r': r, 'sigma': np.nan, 'Price': price_rpv, 'SE': se_rpv})

results

[{'Scenario': 'Vol +100bps',
  'S0': 1235.0,
  'r': 0.12,
  'sigma': 0.21000000000000002,
  'Price': 0.612758960313515,
  'SE': 0.012768608661689584},
 {'Scenario': 'Vol -100bps',
  'S0': 1235.0,
  'r': 0.12,
  'sigma': 0.19,
  'Price': 0.2621891108535402,
  'SE': 0.008060072858695826},
 {'Scenario': 'Rate +100bps',
  'S0': 1235.0,
  'r': 0.13,
  'sigma': 0.2,
  'Price': 0.4705707352602425,
  'SE': 0.011171610927095822},
 {'Scenario': 'Rate -100bps',
  'S0': 1235.0,
  'r': 0.11,
  'sigma': 0.2,
  'Price': 0.3635532861617817,
  'SE': 0.009857702579214474},
 {'Scenario': 'S0 +5%',
  'S0': 1296.75,
  'r': 0.12,
  'sigma': 0.2,
  'Price': 1.2461046753159517,
  'SE': 0.014518055649042497},
 {'Scenario': 'S0 -5%',
  'S0': 1173.25,
  'r': 0.12,
  'sigma': 0.2,
  'Price': 0.11048867244945139,
  'SE': 0.005629658142399387},
 {'Scenario': 'Random-path-vol (σ~N(σ,0.01))',
  'S0': 1235.0,
  'r': 0.12,
  'sigma': nan,
  'Price': 0.4172137611714579,
  'SE': 0.009664842824681979}]

## 8 — results
Combine base row and the stress scenario results into a DataFrame and show diffs vs base.

In [62]:
# base row
base_row = {'Scenario': 'Base', 'S0': S0, 'r': r, 'sigma': sigma, 'Price': price_est, 'SE': stderr}
all_rows = [base_row] + results
df = pd.DataFrame(all_rows)
df['Diff_vs_Base'] = df['Price'] - price_est
df['PctDiff_vs_Base'] = df['Diff_vs_Base'] / price_est * 100.0
pd.options.display.float_format = '{:,.6f}'.format

df

,Scenario,S0,r,sigma,Price,SE,Diff_vs_Base,PctDiff_vs_Base
0,Base,"1,235.000000",0.120000,0.200000,0.414135,0.010569,0.000000,0.000000
1,Vol +100bps,"1,235.000000",0.120000,0.210000,0.612759,0.012769,0.198624,47.961096
2,Vol -100bps,"1,235.000000",0.120000,0.190000,0.262189,0.008060,-0.151946,-36.689970
3,Rate +100bps,"1,235.000000",0.130000,0.200000,0.470571,0.011172,0.056436,13.627325
4,Rate -100bps,"1,235.000000",0.110000,0.200000,0.363553,0.009858,-0.050582,-12.213862
5,S0 +5%,"1,296.750000",0.120000,0.200000,1.246105,0.014518,0.831969,200.893215
6,S0 -5%,"1,173.250000",0.120000,0.200000,0.110489,0.005630,-0.303647,-73.320627
7,"Random-path-vol (σ~N(σ,0.01))","1,235.000000",0.120000,NaN,0.417214,0.009665,0.003079,0.743375
